In [1]:

from brt.routers.inference import make_homo_proto_tensor_cls
make_homo_proto_tensor_cls()

from brt.routers.app import RandHomoFusedScatterRouter
from brt.routers.inference import HomoFusedGatherRouter
import brt.nn as nn
import torch


class FusedMoE(nn.Module):
    def __init__(self, expert_num):
        super().__init__()
        self.scatter_router = RandHomoFusedScatterRouter(
            dst_num=expert_num, supported_capacities=[2, 4, 8, 16, 32, 64]
        )
        self.gather_router = HomoFusedGatherRouter(dst_num=expert_num)

    def forward(self, inputs):
        route_results = self.scatter_router(inputs)
        print(route_results)
        route_results = self.gather_router(route_results)
        return route_results

fused_moe = FusedMoE(expert_num=8).cuda()

input_tensor = torch.rand((64, 16)).cuda()
print(input_tensor)
output_tensor = fused_moe(input_tensor)
print(output_tensor)
print(torch.allclose(output_tensor, input_tensor))


tensor([[0.8963, 0.3466, 0.5708,  ..., 0.6110, 0.7391, 0.4314],
        [0.5459, 0.9828, 0.3737,  ..., 0.3352, 0.0731, 0.5428],
        [0.3865, 0.6751, 0.9694,  ..., 0.1623, 0.0323, 0.6837],
        ...,
        [0.1685, 0.0074, 0.0995,  ..., 0.7589, 0.3094, 0.6204],
        [0.9232, 0.8182, 0.3277,  ..., 0.6840, 0.5562, 0.6488],
        [0.9809, 0.5705, 0.1350,  ..., 0.0521, 0.3849, 0.2090]],
       device='cuda:0')
ProtoTensor(
data: tensor([[0.2577, 0.7951, 0.4149,  ..., 0.1554, 0.3621, 0.8788],
        [0.2746, 0.0862, 0.7195,  ..., 0.4270, 0.9524, 0.8315],
        [0.7696, 0.1514, 0.2461,  ..., 0.4508, 0.4446, 0.0859],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
tag_stack: [tensor([[ 0,  1,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  1,  0,  0,  0],
        [ 0,  0,  1,  0,  0,  0,  0,  0],